In [1]:
# author=cxf
# date=2020-8-8
"""
    file for setting the optimal cutoff for each sample according to the criterion
    1.after filtering error rate less than 0.01 (1/5 of 0.05)
    2.after filtering the number of genotyped sites is the largest and record the number
    (check among points where the error rate less than 0.01)
"""

import numpy as np

i = '../../0.prepare_processing/run1/error.txt'
name = i.split('.')[0]
data4 = []
error = []
name_list=[]
with open(i, 'r') as fx3:
    for line in fx3.readlines():
        each_sample_name = np.array(line[0:-1].split(','))[0]
        each_sample = np.array(line[0:-1].split(','))[1:].astype('float')
        error.append(each_sample)
        name_list.append(each_sample_name)
error = np.array(error)

i = '../../0.prepare_processing/run1/a95.txt'
name = i.split('.')[0]
sites = []
with open(i, 'r') as fx3:
    for line in fx3.readlines():
        each_sample_name = np.array(line[0:-1].split(','))[0]
        each_sample = np.array(line[0:-1].split(','))[1:].astype('int32')
        sites.append(each_sample)
sites = np.array(sites)

f = open(f"95_result.txt", "a+")
print('sample','precise', 'max_cutoff', 'max_num', 'error_rate',sep=',', file=f)
for i in range(error.shape[0]):
    site_info = sites[i]
    error_info = error[i]
    site_info[error_info > 0.01] = 0
    max_index = np.argmax(site_info)
    if error_info[max_index] >= 0.01 or site_info[max_index]==0 :
        error_info[error_info == 0] =1
        error_rate=error_info[np.argmin(error_info)]
    else:
        error_rate=error_info[max_index]
    print(name_list[i],95, max_index, np.max(site_info), error_rate, sep=',', file=f)
f.close()




In [2]:
# take a look 
# to tell you where each output comes from we leave a copy where it generates.
# you may see the max error rate exceeds the criterion which means the least error rate after 1-10 cutoff is here
# and we will discard this sample before following training.
# the cause of the high least error rate may be the contaimination or something else which brings in some errors 
# can not be erased by UMI or removing low RNCU reads.
import pandas as pd 
df=pd.read_csv('95_result.txt',index_col=0)
print(df)
print(df.describe())

           precise  max_cutoff  max_num  error_rate
sample                                             
sample107       95           1      828    0.001893
sample105       95           1      903    0.001320
sample103       95           1      893    0.001222
sample108       95           1      903    0.001091
sample106       95           1      941    0.001302
...            ...         ...      ...         ...
sample91        95           1      924    0.006985
sample95        95           1      922    0.004996
sample97        95           1      888    0.001601
sample98        95           1      952    0.000988
sample99        95           1      867    0.003564

[192 rows x 4 columns]
       precise  max_cutoff      max_num  error_rate
count    192.0  192.000000   192.000000  192.000000
mean      95.0    1.447917   670.489583    0.004432
std        0.0    0.676761   302.014319    0.011849
min       95.0    0.000000     0.000000    0.000312
25%       95.0    1.000000   481.750000 